# Constant pH molecular dynamics in MS$\lambda$D framework

## Introduction

### The objective of this tutorial is to illustrate how to perform constant pH molecular dynamics (CpHMD) simulations using MS$\lambda$D. We will learn:
- ### (1) The physical principle behind CpHMD
- ### (2) Set up a CpHMD simulation for model compound: Lys in water
- ### (3) Set up a CpHMD simulation for a peptide
- ### (4) Analyze the simulation results: get titration curve and pKa
- ### (5) Remarks about advanced techniques

## (1) The physical principle behind CpHMD

<img src="fig/Fig1-thermodynamic-cycle.jpg" width=500/>

<img src="fig/Fig2-pka-eqn.jpg" width=600 />

### According to Eq. 5, if we know the $pK_a$ of model compound, we can use $\Delta\Delta G=\Delta G_{protein}-\Delta G_{model}$ to compute the $pK_a$ of a titratable group in the protein environment. Here we assume that the non-MM part (i.e., quantum effects that can not be described by molecular mechanics) of $\Delta G$ is the same for both legs.

### The potential energy of the system in MS$\lambda$D is as follows. Here, the purpose of using $V_{bias}(\{\lambda\})$ is to flatten the alchemical free energy landscape, such that we can get sufficient sampling of all end states. For instance, for a Lys dipeptide in water, if $V_{bias}(\{\lambda\})$ is the negative of corresponding potential of mean force (PMF), we can obtain effcient sampling in alchemical space in MS$\lambda$D simulations.

<img src='fig/Fig3-msld-energy-1.jpg' width=800 />

A | B
- | - 
![alt](fig/Fig4-lys-free-ener-profile.jpg) | ![alt](fig/Fig5-lys-lmd-trj.jpg)

### The goal of ALF is to obtain optimal $V_{bias}(\{\lambda\})$ using short trial simulations. In MS$\lambda$D framework, there are four types of biasing potential so far. In CpHMD, two of them are more important, the fixed term $V_{fixed}(\{\lambda\})$, and the quadratic term $V_{quadratic}(\{\lambda\})$. The linear term $V_{fixed}(\{\lambda\})$ is to make the free energies of different protonation states the same, and the quadratic term $V_{quadratic}(\{\lambda\})$ is to lower the free energy barrier between different protonation states and it doesn't perturb the end states.

<img src='fig/Fig3-msld-energy-2.jpg' width=800 />

<table><tr>
<td> <img src="fig/fixed.jpg" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="fig/quad.jpg" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

## (2) Set up a CpHMD simulation for model compound: Lys in water

### Here, we will simulate a Lys dipeptide in water with $pH$ equal to its experimental $pK_a$. ALF will be used to optimize the biasing potential $V_{bias}(\{\lambda\})$. Based on the above equation, we can see that when $pH = pK_a$, the pH dependent term in $V_{fixed}(\{\lambda\})$ is zero. To obtain equal population of protonated and unprotonated states, parameters of $V_{fixed}(\{\lambda\})$ obtained from ALF should reflect the pronation free energy of Lys in water (i.e., $\Delta G_{model}$).

### (2.1) Use ALF to obtain optimal $V_{bias}(\{\lambda\})$

In [1]:
# import pycharmm and other necessary modules
import os
import sys
import subprocess
import numpy as np
import pandas as pd
import shlex

import alf
import shutil

import pycharmm
import pycharmm.read as read
import pycharmm.lingo as lingo
import pycharmm.generate as gen
import pycharmm.settings as settings
import pycharmm.write as write
import pycharmm.nbonds as nbonds
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dynb
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.select as select
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.param as param
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.shake as shake
import pycharmm.scalar as scalar
import pycharmm.charmm_file as charmm_file


#### Environmental variables `CHARMM_LIB_DIR` and `ALF_SOURCE_DIR` need to be defined before running this script

In [2]:
pyCHARMM_LIB = '/home/xrliu/program/charmm/stable/charmm-compile/build_pycharmm/install/lib'
if os.getenv('CHARMM_LIB_DIR') == None:
    os.environ['CHARMM_LIB_DIR'] = pyCHARMM_LIB
print(os.getenv('CHARMM_LIB_DIR'))


ALF_SOURCE_DIR = '/home/xrliu/program/ALF'
if os.getenv('ALF_SOURCE_DIR') == None:
     os.environ['ALF_SOURCE_DIR'] = ALF_SOURCE_DIR
print(os.getenv('ALF_SOURCE_DIR'))

/home/xrliu/program/charmm/stable/charmm-compile/build_pycharmm/install/lib
/home/xrliu/program/ALF


#### The following variables need to be defined to reflect information of the system. A particularly important file is prep/alf_info.py. This will be used by ALF.

In [3]:
segid='PROT'   # segment ID for the peptide
tsim=298.15    # simulation temperature

param_dir='./toppar_c36_jul21' # force field parameter directory
wrk_dir='./water'              # working directory
comp='lys'                     # residue name


# The following information is from file prep/alf_info.py
cwd=os.getcwd()
os.chdir(wrk_dir)
alf_info=alf.initialize_alf_info('charmm')
os.chdir(cwd)

name=alf_info['name']
nblocks=alf_info['nblocks']
nsubs=alf_info['nsubs']
nsites=len(alf_info['nsubs'])

# A dictionary to reflect all titratable residues in the system. 
# Keys are residue names, like LYS, ASP, GLU. 
# Values are list of two lists: the first list contains residue IDs; the second list contains residue index, which can be empty.
titr_res={comp[0:3].upper():[[1]]}  

In [4]:
log_unit=30  # unit for CHARMM output file
dcd_unit=40  # unit for dcd trajectory file
rst_unit=50  # unit for restart file
lmd_unit=60  # unit for lambda trajectory file
rpr_unit=70  # unit for previous restart file

# experimental pKa for model compound
pka={'ASP':4.0,
     'GLU':4.4,
     'HSD':7.0,
     'HSE':6.6,
     'LYS':10.40}

In [5]:
def init_variables():
    '''
    A function to initialize biasing potential parameters by setting all parameters to zero.
    '''
    AA=comp[0:3].upper()
    b=np.array([[0,bias[AA+'Gbias']]])
    c=np.zeros([nblocks,nblocks])
    x=c
    s=c
    b_prev=np.zeros([1,nblocks])
    c_prev=c
    x_prev=x
    s_prev=s
    var_init_dir=wrk_dir+'/analysis0'
    os.system('mkdir -p '+var_init_dir+'/../nbshift')
    np.savetxt(var_init_dir+'/b_prev.dat',b_prev)
    np.savetxt(var_init_dir+'/b.dat',b)
    np.savetxt(var_init_dir+'/c_prev.dat',c_prev)
    np.savetxt(var_init_dir+'/c.dat',c)
    np.savetxt(var_init_dir+'/x_prev.dat',x_prev)
    np.savetxt(var_init_dir+'/x.dat',x)
    np.savetxt(var_init_dir+'/s_prev.dat',s_prev)
    np.savetxt(var_init_dir+'/s.dat',s)
    np.savetxt(var_init_dir+'/../nbshift/b_shift.dat',b_prev)
    np.savetxt(var_init_dir+'/../nbshift/c_shift.dat',c_prev)
    np.savetxt(var_init_dir+'/../nbshift/x_shift.dat',x_prev)
    np.savetxt(var_init_dir+'/../nbshift/s_shift.dat',s_prev)


In [6]:
def open_clog():
    '''
    specify charmm output file for a given system
    '''
    clog=charmm_file.CharmmFile(file_name=log_fn,file_unit=log_unit,read_only=False,formatted=True)
    lingo.charmm_script('outu '+str(log_unit))

In [7]:
def read_param():
    '''
    read CHARMM force field parameters.
    note that special RTF file is needed for this tutorial.
    '''
    read.rtf(param_dir+'/top_all36_prot_hedi_xrliu.rtf')
    read.prm(param_dir+'/par_all36m_prot.prm',flex=True)
    lingo.charmm_script('stream '+param_dir+'/toppar_water_ions.str')

In [8]:
def read_init():
    '''
    read psf and initial structure file
    '''
    read.psf_card(f'{wrk_dir}/{comp}-solv.psf')
    if int(run)==1:
       read.pdb(f'{wrk_dir}/{comp}-mini.pdb',resid=True)
    else:
       rst_from=np.loadtxt(f'{out_dir}/../variables/rst-from-{run}.txt',dtype=int)
       read.pdb(f'{out_dir}/../run{rst_from}/pdb/prod.pdb',resid=True)

In [9]:
def titr_grp(resn):
    '''
    atom names in the titratable group for a given amino acid
    '''
    if resn == 'ASP':
       type_list=['CB','HB1','HB2','CG','OD1','OD2','HD1','HD2']
    elif resn == 'GLU':
       type_list=['CG','HG1','HG2','CD','OE1','OE2','HE1','HE2']
    elif resn == 'HSP' or resn == 'HSD' or resn == 'HSE':
       type_list=['CB','HB1','HB2','CG','ND1','HD1','CE1','HE1','CD2','HD2','NE2','HE2']
    elif resn == 'LYS':
       type_list=['CE','HE1','HE2','NZ','HZ1','HZ2','HZ3']
    return type_list



In [10]:
def define_sub(titr_res_dict):
    '''
    define substituents. 
    see the special RTF file for more information.
    '''
    for aa in titr_res_dict.keys():
        resn=aa.upper()
        atom_list=titr_grp(resn)
        for ires in titr_res_dict[resn][0]:
            resid=str(ires)
            w_str=' '.join([i+'W' for i in atom_list])
            m_str=' '.join([i+'M' for i in atom_list])
            p_str=' '.join([i+'P' for i in atom_list])
            u_str=' '.join([i+'U' for i in atom_list])
            sele_w=pycharmm.SelectAtoms().by_res_and_type(seg_id=segid,res_id=resid,atom_type=w_str)
            sele_m=pycharmm.SelectAtoms().by_res_and_type(seg_id=segid,res_id=resid,atom_type=m_str)
            sele_p=pycharmm.SelectAtoms().by_res_and_type(seg_id=segid,res_id=resid,atom_type=p_str)
            sele_u=pycharmm.SelectAtoms().by_res_and_type(seg_id=segid,res_id=resid,atom_type=u_str)
            sele_w.store(f'site{resn}{resid}subW')
            sele_m.store(f'site{resn}{resid}subM')
            sele_p.store(f'site{resn}{resid}subP')
            sele_u.store(f'site{resn}{resid}subU')
            lingo.charmm_script(f'print coor sele site{resn}{resid}subW end')
            lingo.charmm_script(f'print coor sele site{resn}{resid}subM end')
            lingo.charmm_script(f'print coor sele site{resn}{resid}subP end')
            lingo.charmm_script(f'print coor sele site{resn}{resid}subU end')
            


In [11]:
class setup_block:
    '''
    Set up block for CpHMD simulation in the MSLD framework.
    '''
    def __init__(self,bias,titr_res_dict,pH):
        self.Gbias=bias
        self.titr_res_dict=titr_res_dict
        self.pH=pH
        """
        Parameters
        ----------
        Gbias : pandas.DataFrame
             biasing potential parameters
        titr_res_dict : dictionary
             a dictionary to reflect all titratable residues in a system.
             Keys are residue names, like LYS, ASP, GLU. 
             Values are list of two lists: the first list contains residue IDs; the second list contains residue index, which can be empty.
        pH : float
             pH value under which CpHMD simulation is performed
            
        """
    def run(self):
        """
        run the BLOCK command
        """
        self.reset()
        self.call()
    def set_utag(self,resname):
        """
        Depending the amino acid and reference state, use either UNEG or UPOS for the pH-dependent term
        """
        if resname == 'ASP' or resname == 'GLU':
           utag='UNEG'
        elif resname == 'HSD' or resname == 'HSE' or resname == 'HSP' or resname == 'LYS':
           utag='UPOS'
        return utag
    def set_name(self,resname,res_id):
        """
        Based on the speicial RTF file, get a list of selection names for a given residue.
        """
        sele_name_w='site%s%ssubW'%(resname,res_id)
        sele_name_m='site%s%ssubM'%(resname,res_id)
        sele_name_p='site%s%ssubP'%(resname,res_id)
        sele_name_u='site%s%ssubU'%(resname,res_id)
        if resname == 'ASP' or resname == 'GLU':
           sele_name=[sele_name_w,sele_name_m,sele_name_p]
        elif resname == 'HSD':
           sele_name=[sele_name_w,sele_name_u]
        elif resname == 'HSE':
           sele_name=[sele_name_w,sele_name_m]
        elif resname == 'LYS':
           sele_name=[sele_name_w,sele_name_m]
        elif resname == 'HSP':
           sele_name=[sele_name_w,sele_name_u,sele_name_m]
        return sele_name
    def set_pka(self,resname):
        if resname == 'HSP':
           refpka_list=[pka['HSD'],pka['HSE']]
        else:
           refpka_list=[pka[resname],pka[resname]]
        return refpka_list
    def reset(self):
        """
        clear BLOCK
        """
        lingo.charmm_script('''
              BLOCK {}
                    clear 
              END
        '''.format(nblocks+1))
    def call(self):
        '''
        Generate a string that will be used by lingo.charmm_script().
        
        note: don't put these commands into multiple block sections.
        otherwise energy can be incorrect in domdec and blade.
        '''

        block_str=f'BLOCK {nblocks+1}\n'
        ldin_str='      !   id lmd vel mass energy friction-coefficient\n'
        ldin_str+='      ldin 1 1.0000  0.0  12.0     0.00  5.0\n'
        msld_str='      msld 0 '

        iblock=2
        isite=0
        for aa in self.titr_res_dict.keys():
            resn=aa.upper()
            for ires in self.titr_res_dict[resn][0]:
                resid=str(ires)
                utag=self.set_utag(resn)
                refpka=self.set_pka(resn)
                sele_name_lst=self.set_name(resn,resid)
                for isub in range(nsubs[isite]):
                    #print(isite,isub)
                    sele_name=sele_name_lst[isub]
                    if isub==0:
                        phtag='NONE'
                    else:
                        phtag=utag+' '+str(refpka[isub-1])
                    block_str+=f'      call {iblock} sele {sele_name} end\n'
                    ldin_str+='      ldin {} {:.4f}  0.0  12.0  {:7.2f}  5.0\n'.format(iblock,1/nsubs[isite],self.Gbias.loc[f'lams{isite+1}s{isub+1}','value'])
                    
                    msld_str+=f'{isite+1} '
                    iblock+=1
                isite+=1
        msld_str+='fnex 5.5\n'

        excl_lst='      excl '
        iblock=2
        for isite in range(nsites):
            for isub in range(nsubs[isite]):
                shift=1
                for jsub in range(isub+1,nsubs[isite]):
                    excl_lst+='{} {} '.format(iblock,iblock+shift)
                    shift+=1
                iblock+=1
        excl_lst+='\n'

        nldbi=int(5*nblocks*(nblocks-1)/2)
        ldbi_str='      ldbi {}\n'.format(nldbi)
        ibias=0
        iblock0=2
        for isite in range(nsites):
            jblock0=iblock0
            for jsite in range(isite,nsites):
                for isub in range(nsubs[isite]):
                    iblock=iblock0+isub
                    if jsite==isite:
                       jlow=isub+1
                    else:
                       jlow=0
                    for jsub in range(jlow,nsubs[jsite]):
                        jblock=jblock0+jsub
                        ibias+=1
                        ldbi_str+='      ldbv {:3d} {:3d} {:3d}  6   0.0 {:7.2f} 0\n'.format(ibias,iblock,jblock,self.Gbias.loc[f'cs{isite+1}s{isub+1}s{jsite+1}s{jsub+1}','value'])
                        ibias+=1
                        ldbi_str+='      ldbv {:3d} {:3d} {:3d} 10 -5.56 {:7.2f} 0\n'.format(ibias,iblock,jblock,self.Gbias.loc[f'xs{isite+1}s{isub+1}s{jsite+1}s{jsub+1}','value'])
                        ibias+=1
                        ldbi_str+='      ldbv {:3d} {:3d} {:3d} 10 -5.56 {:7.2f} 0\n'.format(ibias,jblock,iblock,self.Gbias.loc[f'xs{jsite+1}s{jsub+1}s{isite+1}s{isub+1}','value'])
                        ibias+=1
                        ldbi_str+='      ldbv {:3d} {:3d} {:3d}  8 0.017 {:7.2f} 0\n'.format(ibias,iblock,jblock,self.Gbias.loc[f'ss{isite+1}s{isub+1}s{jsite+1}s{jsub+1}','value'])
                        ibias+=1
                        ldbi_str+='      ldbv {:3d} {:3d} {:3d}  8 0.017 {:7.2f} 0\n'.format(ibias,jblock,iblock,self.Gbias.loc[f'ss{jsite+1}s{jsub+1}s{isite+1}s{isub+1}','value'])
                jblock0+=nsubs[isite]
            iblock0+=nsubs[isite]

        soft_str='\n'

        block_str+='\n'
        block_str+=excl_lst
        block_str+='\n'
        block_str+='      qldm theta\n'
        block_str+='      lang temp {}\n'.format(tsim)
        block_str+='      pmel ex\n'
        block_str+='      phmd pH {}'.format(self.pH)
        block_str+=soft_str
        block_str+='\n'
        block_str+=ldin_str
        block_str+='\n'
        block_str+='      rmla bond thet impr\n'
        block_str+=msld_str
        block_str+='      msma\n'
        block_str+='\n'
        block_str+=ldbi_str
        block_str+='END\n'

        lingo.charmm_script(block_str)


In [12]:
# Ensure that FFT grid is product of small primes (2, 3, 5)
def find_prime_prod(n):
    '''
    find a minimum integer that is a product of 2, 3, or 5, and is greater than the given value n
    '''
    n=np.ceil(n)
    find=0
    while find == 0:
          if (n%2 != 0):
             find=0   # we have to find an even number
          else:
             ni=n
             while ni:
                   #print(n,ni)
                   flag=0
                   for x in (2,3,5):
                       if ni % x == 0:
                          ni = ni / x
                          flag=1
                          # "break" terminates the current for/while loop
                          # and resumes execution at the next statement
                          break
                   if ni==1:
                      find=1
                      return int(n)
                   else:
                      if flag == 1:
                         # "continue" rejects the remaining statement in the current iteration
                         # and returns the control to the beginning of the while loop
                         continue
                      break
          n += 1
    return int(n)



nbond={'elec': True,
       'atom': True,
       'cdie': True,
       'eps': 1,
       'vdw': True,
       'vatom': True,
       'vfswitch': True,
       'cutnb': 15,
       'cutim': 15,
       'ctofnb': 12,
       'ctonnb': 10,
       'ewald': True,
       'pmewald': True,
       'kappa': 0.320,
       'order': 6
      }

def setup_nb(nb):
    """
    set up nonbonded interactions
    """
    if int(run)==1:
       size=np.loadtxt(f'{wrk_dir}/{comp}-size.dat')
    else:
       rst_from=np.loadtxt(f'{out_dir}/../variables/rst-from-{run}.txt',dtype=int)
       size=np.loadtxt(f'{out_dir}/../run{rst_from}/size.dat',usecols=(0))    
    fft=find_prime_prod(size)
    nbond['fftx']=fft
    nbond['ffty']=fft
    nbond['fftz']=fft
    nb_pme=pycharmm.NonBondedScript(**nb)
    nb_pme.run()
    #lingo.charmm_script('energy blade')


In [13]:
def set_pbc():
    if int(run)==1:
       size=np.loadtxt(f'{wrk_dir}/{comp}-size.dat')
    else:
       rst_from=np.loadtxt(f'{out_dir}/../variables/rst-from-{run}.txt',dtype=int)
       size=np.loadtxt(f'{out_dir}/../run{rst_from}/size.dat',usecols=(0))
    crystal.define_cubic(size)
    crystal.build(12)
    offset=0
    image.setup_residue(offset,offset,offset,'TIP3')
    image.setup_segment(offset,offset,offset,segid)



In [14]:
def mini():
    '''
    In my initial structure, the restraining atoms are on top of each other.
    I use energy minimization to make their coordinates slightly different, which is required for later
    noe restraint if default charmm is used.
    '''
    minimize.run_abnr(nstep=200,tolenr=1e-3,tolgrd=1e-3)
    energy.show()
    write.coor_pdb(out_dir+'/../mini.pdb')


In [15]:
class scat_restrain:
    '''
    restrain all or heavy atoms between analogous groups of the 2 or 3 protonation states
    using SCAT restraint.
    '''
    def __init__(self,titr_res_dict,heavy=True):
        """
        Parameters
        ----------
        titr_res_dict : dictionary
             a dictionary to reflect all titratable residues in a system.
             Keys are residue names, like LYS, ASP, GLU. 
             Values are list of two lists: the first list contains residue IDs; the second list contains residue index, which can be empty.
        heavy : bool
             True for restraining all heavy atoms.
             Flase for restraining all atoms including H atoms.
        """            
        self.titr_res_dict=titr_res_dict
        self.heavy=heavy
    def restrain(self):
        """
        Generate a string of BLOCK commands for lingo.charmm_script() to apply the SCAT restraining potential.
        """
        scat_str='BLOCK\n'
        scat_str+='      ! enables Scaling of Constrained AToms\n'
        scat_str+='      scat on\n'
        scat_str+='      scat k 300\n'
        self.cats_str=''

        isite=0
        for aa in self.titr_res_dict.keys():
            resn=aa.upper()
            atom_list=titr_grp(resn)
            for ires in self.titr_res_dict[resn][0]:
                resid=str(ires)
                for atom_name in atom_list:
                    if self.heavy==True:  # restrain heavy atoms
                       if atom_name[0]!='H':
                          self.apply_restrain(resid,atom_name)
                    else:                 # restrain all atoms
                       self.apply_restrain(resid,atom_name)

                isite+=1

        #print(self.cats_str)
        scat_str+=self.cats_str
        scat_str+='END\n'
        lingo.charmm_script(scat_str)

    def apply_restrain(self,res_id,name):
        sel_w=pycharmm.SelectAtoms().by_res_and_type(seg_id=segid,res_id=res_id,atom_type=name+'W')
        sel_m=pycharmm.SelectAtoms().by_res_and_type(seg_id=segid,res_id=res_id,atom_type=name+'M')
        sel_p=pycharmm.SelectAtoms().by_res_and_type(seg_id=segid,res_id=res_id,atom_type=name+'P')
        sel_u=pycharmm.SelectAtoms().by_res_and_type(seg_id=segid,res_id=res_id,atom_type=name+'U')
        w=sel_w.get_n_selected()
        m=sel_m.get_n_selected()
        p=sel_p.get_n_selected()
        u=sel_u.get_n_selected()
        if w+m+p+u>=2:  # this is required when blade is used, otherwise we will see errors like "Error running curandGenerateNormal(gen, p, n, 0.0f, 1.0f) in file source/blade/src/rng/rng_gpu.cxx, function rand_normal, line 89, Error string: CURAND_STATUS_LAUNCH_FAILURE" 
           self.cats_str+='      cats sele segid {} .and. resid {} .and. ( type {}W .or. type {}M .or. type {}P .or. type {}U ) end\n'.format(segid, res_id, name, name, name, name)



In [16]:
def dyn_init():
    lingo.charmm_script('''
          faster on
          blade on
          shake fast bonh param
    ''')
    n = psf.get_natom()
    scalar.set_fbetas([1.0] * n)



In [17]:
def set_var(Step):
    """
    After using ALF to determine how much biasing potential parameters should change,
    calcuate new biasing potential parameters that will be used the next run,
    and save the corresponding pd.DataFrame to a file.
    """
    nblocks=np.loadtxt('../prep/nblocks',dtype='int')
    nsubs=np.loadtxt('../prep/nsubs',dtype='int',ndmin=1)
    nreps=np.loadtxt('../prep/nreps',dtype='int')
    nnodes=np.loadtxt('../prep/nnodes',dtype='int')
    ncentral=np.loadtxt('../prep/ncentral',dtype='int')
    
    var_dict={}
    
    b_prev=np.loadtxt('b_prev.dat')
    b=np.loadtxt('b.dat')
    b_sum=b_prev+b
    b_sum=np.reshape(b_sum,(1,-1))
    np.savetxt('b_sum.dat',b_sum,fmt=' %7.2f')
    
    ibuff=0
    for i in range(0,len(nsubs)):
      for j in range(0,nsubs[i]):
        var_dict['lams%ds%d'%(i+1,j+1)]=b_sum[0,ibuff+j]
      ibuff+=nsubs[i]
    
    c_prev=np.loadtxt('c_prev.dat')
    c=np.loadtxt('c.dat')
    c_sum=c_prev+c
    np.savetxt('c_sum.dat',c_sum,fmt=' %7.2f')

    ibuff=0
    for si in range(0,len(nsubs)):
      jbuff=ibuff
      for sj in range(si,len(nsubs)):
        for i in range(0,nsubs[si]):
          ii=i+ibuff
          j0=0
          if si==sj:
            j0=i+1
          for j in range(j0,nsubs[sj]):
            jj=j+jbuff
            var_dict['cs%ds%ds%ds%d'%(si+1,i+1,sj+1,j+1)]=-c_sum[ii,jj]
        jbuff+=nsubs[sj]
      ibuff+=nsubs[si]
    
    x_prev=np.loadtxt('x_prev.dat')
    x=np.loadtxt('x.dat')
    x_sum=x_prev+x
    np.savetxt('x_sum.dat',x_sum,fmt=' %7.2f')
    
    s_prev=np.loadtxt('s_prev.dat')
    s=np.loadtxt('s.dat')
    s_sum=s_prev+s
    np.savetxt('s_sum.dat',s_sum,fmt=' %7.2f')
    
    ibuff=0
    for si in range(0,len(nsubs)):
      jbuff=0
      for sj in range(0,len(nsubs)):
        for i in range(0,nsubs[si]):
          ii=i+ibuff
          for j in range(0,nsubs[sj]):
            jj=j+jbuff
            if ii!=jj:
              var_dict['xs%ds%ds%ds%d'%(si+1,i+1,sj+1,j+1)]=-x_sum[ii,jj]
              var_dict['ss%ds%ds%ds%d'%(si+1,i+1,sj+1,j+1)]=-s_sum[ii,jj]
        jbuff+=nsubs[sj]
      ibuff+=nsubs[si]
    
    # save variables 
    var_df=pd.DataFrame.from_dict(var_dict, orient='index')
    
    var_df.to_csv('../variables/var-run'+str(Step)+'.txt',header=['value'])


In [18]:
def alf_analysis(i,dir='.'):
    """
    A function to call ALF functions to determine how biasing potential parameters should be updated
    based on exisiting short trial simulations.
    """
    cwd=os.getcwd()
    os.chdir(dir)
    
    # Prep the analysis directories
    print('analysis%d started' % i)
    if not os.path.exists('analysis%d' % i):
      os.mkdir('analysis%d' % i)
    shutil.copy('analysis%d/b_sum.dat' % (i-1),'analysis%d/b_prev.dat' % i)
    shutil.copy('analysis%d/c_sum.dat' % (i-1),'analysis%d/c_prev.dat' % i)
    shutil.copy('analysis%d/x_sum.dat' % (i-1),'analysis%d/x_prev.dat' % i)
    shutil.copy('analysis%d/s_sum.dat' % (i-1),'analysis%d/s_prev.dat' % i)
    if not os.path.exists('analysis%d/G_imp' % i):
      G_imp_dir=os.getenv('ALF_SOURCE_DIR')+'/v3.2beta/alf/G_imp'
      print(G_imp_dir)
      os.symlink(G_imp_dir,'analysis%d/G_imp' % i)
    os.chdir('analysis%d' % i)
    
    # Run the analysis
    
    im5=max(i-4,1)
    N=i-im5+1
    alf.GetLambdas(alf_info,i)
    alf.GetEnergy(alf_info,im5,i)
    fpout=open('output','w')
    fperr=open('error','w')
    subprocess.call([shutil.which('python'),'-c','import alf; alf.RunWham(%d,0,0)' % (N*alf_info['nreps'])],stdout=fpout,stderr=fperr) # `cat ../ntersiteflat`
    print('Warning: coupling flags ignored')
    alf.GetFreeEnergy5(alf_info,0,0) # `cat ../ntersiteflat`

    set_var(i+1)
    os.system(f'echo {1+np.random.randint(i)} > ../variables/rst-from-{i+1}.txt')

    
    os.chdir(cwd)



In [19]:
def clear_system():
    """
    clear CHARMM system to make sure we can iterate simulations in ALF.
    """
    psf.delete_atoms(pycharmm.SelectAtoms(select_all=True))
    lingo.charmm_script('crystal free')

#### Below is the main script to run short trial simulations and use ALF to do analysis.

In [20]:
phase=1        # phase in ALF flattening
itt=None       # index for iteraction in production run
alpha=None     # an alphabet for one of the independent runs in production

for run in range(1,35):
    if not os.path.exists(f'{wrk_dir}/analysis{run}/b_sum.dat'):
        if int(phase)==1 or int(phase)==2:
           if int(phase)==1:
              nsteps_eq=10000    # 20 ps
              nsteps_prod=40000  # 80 ps
           elif int(phase)==2:
              nsteps_eq=125000   # 250 ps
              nsteps_prod=375000 # 750 ps
           nsavc=1000         # saving frequency for dcd file
           out_dir=f'{wrk_dir}/run{run}'  # output directory
           log_fn=f'{out_dir}/prod.out'   # CHARMM output/log file
        elif int(phase)==3:
           nsteps_eq=0
           nsteps_prod=500000 # 1 ns
           nsavc=10000   # saving frequency for dcd file
           ittm1=itt-1
           out_dir=f'{wrk_dir}/run{run}{alpha}' # output directory
           log_fn=f'{out_dir}/prod{itt}.out'    # CHARMM output/log file

        os.system('mkdir -p '+out_dir+'/dcd')
        os.system('mkdir -p '+out_dir+'/res')
        os.system('mkdir -p '+out_dir+'/pdb')

        open_clog()
        read_param()
        
        read_init()
        
        set_pbc()
        define_sub(titr_res)
        
        if int(run)==1:
           init_variables()
           set_variables()
        var_df=pd.read_csv(f'{wrk_dir}/variables/var-run{run}.txt',index_col=0,header=0)
        blk=setup_block(var_df,titr_res,pka[comp[0:3].upper()])
        blk.run()
        setup_nb(nbond)
        
        #myscat=scat_restrain(comp[0:3],1,heavy=False)
        myscat=scat_restrain(titr_res,heavy=True)
        myscat.restrain()
        
        lingo.charmm_script('energy blade')

        # NPT simulation
        dyn_dict={
            'leap': True,
            'verlet': False,
            'cpt': False,
            'new': False,
            'langevin': False,
            'omm': False,
            'blade': True,
            'start': True,
            'timestep': 0.002,
            'nstep': nsteps_eq,
            'nsavc': nsavc,
            'nsavv': 0,
            'nsavl': 100,    # frequency for saving lambda values in lamda-dynamics
            'nprint': nsavc, # Frequency to write to output
            'iprfrq': nsavc, # Frequency to calculate averages
            'isvfrq': nsavc, # Frequency to save restart file
            'ntrfrq': 0,
            'inbfrq':-1,
            'ihbfrq':0,
            'ilbfrq':0,
            'imgfrq':-1,
            'iunrea':-1,
            'iunwri':rst_unit,
            'iuncrd':dcd_unit,
            'iunldm':lmd_unit,
            'firstt': tsim,
            'finalt': tsim,
            'tstruct': tsim,
            'tbath': tsim,
            'prmc': True,
            'iprs': 10,  # Monte carlo pressure frequency (50 is default)
            'pref': 1,   # reference pressure (atmospheres)
            'prdv': 100, # RMS proposed volume change (A cubed)
            'iasors': 1, # assign velocities
            'iasvel': 1, # method for assignment of velocities during heating & equil when IASORS is nonzero.
                         # This option also controls the initial assignment of velocities 
            'iscale': 0, # not scale velocities on a restart
            'scale': 1,  # scaling factor for velocity scaling
            'ichecw': 0, # not check temperature
            'echeck': -1 # not check energy
        }
        dyn_init()
        
        # equilibration
        if int(phase)==1 or int(phase)==2:
           dcd_fn=out_dir+'/dcd/'+name+'_eq.dcd'
           rst_fn=out_dir+'/res/'+name+'_eq.res'
           lmd_fn=out_dir+'/res/'+name+'_eq.lmd'
           dcd=charmm_file.CharmmFile(file_name=dcd_fn,file_unit=dcd_unit,read_only=False,formatted=False)
           rst=charmm_file.CharmmFile(file_name=rst_fn,file_unit=rst_unit,read_only=False,formatted='formatted')
           lmd=charmm_file.CharmmFile(file_name=lmd_fn,file_unit=lmd_unit,read_only=False,formatted=False)
           npt_eq=pycharmm.DynamicsScript(**dyn_dict)
           npt_eq.run()
           dcd.close()
           rst.close()
           lmd.close()
        
        
        # production
        if int(phase)==1 or int(phase)==2:
           dcd_fn=out_dir+'/dcd/'+name+'_flat.dcd'
           rst_fn=out_dir+'/res/'+name+'_flat.res'
           lmd_fn=out_dir+'/res/'+name+'_flat.lmd'
        elif int(phase)==3:
           dcd_fn=out_dir+'/dcd/'+name+'_prod'+str(itt)+'.dcd'
           rst_fn=out_dir+'/res/'+name+'_prod'+str(itt)+'.res'
           lmd_fn=out_dir+'/res/'+name+'_prod'+str(itt)+'.lmd'
           if itt>=2:
              dyn_dict['start']=False
              dyn_dict['restart']=True
              dyn_dict['iunrea']=rpr_unit
              rpr_fn=out_dir+'/res/'+name+'_prod'+str(ittm1)+'.res'
              rpr=charmm_file.CharmmFile(file_name=rpr_fn,file_unit=rpr_unit,read_only=True,formatted='formatted')
        
        dyn_dict['nstep']=nsteps_prod
        dcd=charmm_file.CharmmFile(file_name=dcd_fn,file_unit=dcd_unit,read_only=False,formatted=False)
        rst=charmm_file.CharmmFile(file_name=rst_fn,file_unit=rst_unit,read_only=False,formatted='formatted')
        lmd=charmm_file.CharmmFile(file_name=lmd_fn,file_unit=lmd_unit,read_only=False,formatted=False)
        npt_prod=pycharmm.DynamicsScript(**dyn_dict)
        npt_prod.run()
        dcd.close()
        rst.close()
        lmd.close()
        
        # box size
        boxx=lingo.get_energy_value('XTLA')
        boxy=lingo.get_energy_value('XTLB')
        boxz=lingo.get_energy_value('XTLC')
        size=np.array([[boxx,boxy,boxz]])
                
        # last snapshot
        if int(phase)==1 or int(phase)==2:
           write.coor_pdb(out_dir+'/pdb/prod.pdb')
           np.savetxt(out_dir+'/size.dat',size)
        elif int(phase)==3:
           write.coor_pdb(out_dir+'/pdb/prod'+str(itt)+'.pdb')
           np.savetxt(out_dir+'/size'+str(itt)+'.dat',size)
        
        # analyze l-dynamics output
        lmd=charmm_file.CharmmFile(file_name=lmd_fn,file_unit=lmd_unit,read_only=True,formatted=False)
        lingo.charmm_script('traj lamb print ctlo 0.95 cthi 0.99 first %s nunit 1'%(lmd_unit))
        lmd.close()
        
        # run ALF analysis to update biasing potential parameters
        alf_analysis(int(run),dir=wrk_dir)

        # clear CHARMM systems for the next run
        clear_system()

 VOPEN> Attempting to open::./water/run33/prod.out::
  
 CHARMM>     outu 30
A selection has been stored as SITELYS1SUBW
A selection has been stored as SITELYS1SUBM
A selection has been stored as SITELYS1SUBP
A selection has been stored as SITELYS1SUBU
Creating a copy of system
allocating kcmap=0x55609f968740
allocating kcmap=0x55609f968720
allocating kcmap=0x55609f968720
allocating kcmap=0x55609f968720
allocating kcmap=0x55609f968720
allocating kcmap=0x55609f968720
PME grid(0) size: 36
PME grid(1) size: 36
PME grid(2) size: 36
The following parameters are set heuristically at /home/xrliu/program/charmm/stable/charmm-compile/source/blade/src/domdec/domdec.cu:109, and can cause errors if set too low
maxBlocks=466
maxPartnersPerBlock=834
freqDomdec=10 (how many steps before domain reset)
cullPad=1.73008 (spatial padding for considering which blocks could interact
maxBlockExclCount=1243 (can reallocate dynamically with "run setvariable domdecheuristic off")
Destroying a copy of system
Cre

/home/xrliu/program/anaconda/anaconda3-2021/envs/CHARMM/lib/python3.11/site-packages/scipy/io/_fortran.py:255: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  dtypes = tuple(np.dtype(dtype) for dtype in dtypes)


scaling is:
1
A selection has been stored as SITELYS1SUBW
A selection has been stored as SITELYS1SUBM
A selection has been stored as SITELYS1SUBP
A selection has been stored as SITELYS1SUBU
Destroying a copy of system
Creating a copy of system
allocating kcmap=0x55609fd8a380
allocating kcmap=0x55609fd8a360
allocating kcmap=0x55609fd8a360
allocating kcmap=0x55609fd8a360
allocating kcmap=0x55609fd8a360
allocating kcmap=0x55609fd8a360
PME grid(0) size: 36
PME grid(1) size: 36
PME grid(2) size: 36
The following parameters are set heuristically at /home/xrliu/program/charmm/stable/charmm-compile/source/blade/src/domdec/domdec.cu:109, and can cause errors if set too low
maxBlocks=466
maxPartnersPerBlock=828
freqDomdec=10 (how many steps before domain reset)
cullPad=1.73008 (spatial padding for considering which blocks could interact
maxBlockExclCount=1243 (can reallocate dynamically with "run setvariable domdecheuristic off")
Destroying a copy of system
Creating a copy of system
allocating k

/home/xrliu/program/anaconda/anaconda3-2021/envs/CHARMM/lib/python3.11/site-packages/scipy/io/_fortran.py:255: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  dtypes = tuple(np.dtype(dtype) for dtype in dtypes)


scaling is:
1


### (2.2) Validation: use optimal $V_{bias}(\{\lambda\})$ from ALF to run a CpHMD simulation (optional but recommended) 

## (3) Set up a CpHMD simulation for a peptide

## (4) Analyze the simulation results: get titration curve and pKa

## (5) Advanced techniques